# 数据分区

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)

In [2]:
data = pd.read_csv('data_keep_50000.csv', low_memory=False)

In [3]:
data.shape

(57298, 257)

一共256个特征，其中有11个是数值，245个是混合类型，需要处理。

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57298 entries, 0 to 57297
Columns: 257 entries, vid to A601
dtypes: float64(11), object(246)
memory usage: 112.3+ MB


### 数值数据的分布

In [5]:
data.describe()

,100005,100007,315,316,317,319,33,34,37,39,809009
count,19422.000000,22608.000000,26600.000000,26600.000000,26229.000000,26600.000000,26143.000000,21004.000000,26600.000000,21216.000000,7333.000000
mean,18.967235,0.671348,90.613117,30.289935,334.470525,214.974899,2.076035,0.427824,57.626687,7.709535,1.618898
std,12.079368,4.093520,6.045505,2.474706,15.035377,55.625658,0.733745,0.209185,13.444135,6.317230,0.795881
min,0.000000,0.000000,53.200000,10.300000,116.000000,28.000000,0.200000,0.060000,0.310000,0.010000,0.370000
25%,12.100000,0.159000,87.800000,29.300000,325.000000,178.000000,1.600000,0.300000,53.400000,4.800000,1.400000
50%,13.000000,0.201000,91.000000,30.500000,334.000000,212.000000,2.000000,0.400000,59.300000,6.700000,1.480000
75%,15.000000,0.259000,94.200000,31.700000,344.000000,249.000000,2.400000,0.500000,65.000000,8.700000,1.560000
max,56.800000,51.800000,137.900000,47.600000,485.000000,833.000000,15.600000,4.100000,92.200000,59.000000,21.590000


### 文本数据的分布

**有部分混合数据被分为object，需要再处理**

### 训练数据

In [6]:
train_df = pd.read_csv('meinian_round1_train_20180408.csv')
train_df.shape

(38199, 6)

In [7]:
train_df.describe(include='O')

,vid,收缩压,舒张压,血清甘油三酯
count,38199,38199,38199,38199
unique,38199,156,110,975
top,d9632ea47f425acdd7eed79f9ea17491,120,80,0.8
freq,1,1260,1747,362


有三列数据有混合类型，需要处理。

In [8]:
train_df[train_df['收缩压'].str.isnumeric()==False]

,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
15756,1773a52f313d62c3f8d8b8eec1bd63b9,未查,未查,1.15,1.21,3.91
19645,1c1773e341fd21a8da7ac51223fc5b04,未查,未查,1.15,1.18,2.03
20652,8eee83b48c083e6e337c32afe5fec05f,未查,未查,0.47,0.78,1.08
20953,85b341038cfa4b03e6c625ecc2cefddb,弃查,弃查,1.13,1.43,2.92
21662,0597838c42f7c667f0b941efd42055a6,弃查,弃查,1.38,1.39,2.95
27521,34d2cd11dd80947d7a5ef1a42e4777bd,未查,未查,1.25,1.52,5.01
32672,eb16145ab9d131a8501f5df67586f3a6,未查,未查,1.63,1.39,3.79


In [9]:
train_df['收缩压'] = pd.to_numeric(train_df['收缩压'], errors='coerce')
train_df['舒张压'] = pd.to_numeric(train_df['舒张压'], errors='coerce')

In [10]:
train_df.describe(include='O')

,vid,血清甘油三酯
count,38199,38199
unique,38199,975
top,d9632ea47f425acdd7eed79f9ea17491,0.8
freq,1,362


In [11]:
train_df[train_df['血清甘油三酯'].str.contains(r'[\u4e00-\u9fa5]')]

,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
22712,99848f481d731504457442dc79e95eee,133.0,88.0,7.75轻度乳糜,1.1,2.06


In [12]:
train_df.loc[22712, '血清甘油三酯'] = 7.75
train_df['血清甘油三酯'] = pd.to_numeric(train_df['血清甘油三酯'], errors='coerce')

In [13]:
train_df.describe()

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
count,38192.000000,38192.000000,38158.000000,38199.000000,38199.000000
mean,126.049618,79.642229,1.612536,1.406683,2.769719
std,19.275625,512.318203,1.335518,0.341184,0.852217
min,0.000000,0.000000,0.100000,0.150000,-1.220000
25%,112.000000,68.000000,0.880000,1.160000,2.180000
50%,124.000000,76.000000,1.270000,1.350000,2.690000
75%,137.000000,85.000000,1.900000,1.600000,3.260000
max,252.000000,100164.000000,28.800000,4.780000,11.540000


In [14]:
train_df['舒张压'].sort_values(ascending=False)[:5]

22357    100164.0
4600        974.0
35452       148.0
33501       146.0
22516       144.0
Name: 舒张压, dtype: float64

In [15]:
train_df.loc[22357,:]

vid         7685d48685028a006c84070f68854ce1
收缩压                                      180
舒张压                                   100164
血清甘油三酯                                     4
血清高密度脂蛋白                                   2
血清低密度脂蛋白                                2.99
Name: 22357, dtype: object

数据不合理，把舒张压替换成NaN

In [16]:
train_df.loc[22357, '舒张压'] = np.nan

In [17]:
train_df['舒张压'].sort_values()[:5]

29394     0.0
5727     37.0
14966    38.0
13674    39.0
15837    40.0
Name: 舒张压, dtype: float64

In [18]:
train_df.loc[29394, '舒张压'] = np.nan

In [19]:
train_df['收缩压'].sort_values()[:5]

29394     0.0
5727     69.0
28325    73.0
5702     74.0
15053    75.0
Name: 收缩压, dtype: float64

In [20]:
train_df.loc[29394, '收缩压'] = np.nan

In [21]:
train_df['血清甘油三酯'].sort_values(ascending=False)[:5]

17693    28.80
3728     25.03
6134     25.01
13137    23.58
30922    22.55
Name: 血清甘油三酯, dtype: float64

In [22]:
train_df['血清低密度脂蛋白'].sort_values()[:5]

11688   -1.22
14065   -0.12
15215   -0.06
36423    0.08
9598     0.09
Name: 血清低密度脂蛋白, dtype: float64

In [23]:
train_df.describe()

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
count,38191.000000,38190.000000,38158.000000,38199.000000,38199.000000
mean,126.052918,77.023619,1.612536,1.406683,2.769719
std,19.265082,13.385752,1.335518,0.341184,0.852217
min,69.000000,37.000000,0.100000,0.150000,-1.220000
25%,112.000000,68.000000,0.880000,1.160000,2.180000
50%,124.000000,76.000000,1.270000,1.350000,2.690000
75%,137.000000,85.000000,1.900000,1.600000,3.260000
max,252.000000,974.000000,28.800000,4.780000,11.540000


In [24]:
test_df = pd.read_csv('meinian_round1_test_a_20180409.csv')
test_df.shape

(9538, 6)

### 数据拼接

需要把训练集和测试集数据分别拼接起来。

In [25]:
merged_train_df = pd.merge(train_df, data, on='vid', sort=False)

In [26]:
merged_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38199 entries, 0 to 38198
Columns: 262 entries, vid to A601
dtypes: float64(16), object(246)
memory usage: 76.6+ MB


In [27]:
merged_test_df = pd.merge(test_df, data, on='vid', sort=False)

In [28]:
merged_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9538 entries, 0 to 9537
Columns: 262 entries, vid to A601
dtypes: float64(16), object(246)
memory usage: 19.1+ MB


因为在数据导入的时候，每一列只要有非数值数据，就会被转化成 object 类型。  
所以需要统计每一列 object 类型的数据的数值数据比例，大于某值（如0.8）则转化成数值，不能转化的变成 NaN。

In [29]:
combine = [merged_train_df, merged_test_df]

numerical_feature = []
train_data_counts = merged_train_df.shape[0]

for col in merged_train_df.columns.values:
    num_counts = merged_train_df[col].astype(
        str).str.match(r'^(-?\d+)(\.\d+)?$').sum()
    na_counts = merged_train_df[col].isna().sum()

    if num_counts / (train_data_counts - na_counts) > 0.8:
        numerical_feature.append(col)

for df in combine:
    df[numerical_feature[5:]] = df[numerical_feature[5:]].apply(
        lambda x: pd.to_numeric(x, downcast='float', errors='coerce'))

### 异常数据点的处理

In [30]:
merged_train_df.describe()

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,0424,100005,100006,100007,100014,10002,10003,10004,1106,1107,1115,1117,1127,1321,1322,1325,1326,1345,139,143,1474,155,1814,1815,183,1840,1845,1850,190,191,192,193,2174,2333,2372,2403,2404,2405,2406,2420,269003,269004,269005,269006,269007,269008,269009,269010,269011,269012,269013,269014,269015,269016,269017,269018,269019,269020,269021,269022,269023,269024,269025,300017,300021,31,312,313,314,315,316,317,319,3193,32,320,33,34,37,38,39,669001,669002,669006,809001,809009,979001,979002,979003,979004,979005,979006,979007,979008,979009,979011,979012,979013,979014,979015,979016,979017,979018,979019,979020,979021,979022,979023
count,38191.000000,38190.000000,38158.000000,38199.000000,38199.000000,25078.000000,12899.000000,17472.000000,15043.000000,4932.000000,20102.000000,18402.000000,34709.000000,5606.000000,5479.000000,22944.000000,26053.000000,8201.000000,16549.000000,16546.000000,6121.000000,6113.000000,8733.000000,5641.000000,5415.000000,5056.000000,8770.000000,37919.000000,32801.000000,20697.000000,36650.000000,17139.000000,35070.000000,37450.000000,37039.000000,29782.000000,25778.000000,20199.000000,17476.000000,17331.000000,36481.000000,36482.000000,36316.000000,13074.000000,9756.000000,8217.000000,8361.000000,8057.000000,8218.000000,7287.000000,8218.000000,8218.000000,8218.000000,7744.000000,8217.000000,8217.000000,8214.000000,8358.000000,8214.000000,8218.000000,8362.000000,8218.000000,8362.000000,8361.000000,8361.000000,8361.000000,8361.000000,8315.000000,4904.000000,5754.000000,17712.000000,17539.000000,17458.000000,20556.000000,17712.000000,17712.000000,17459.000000,17712.000000,36557.000000,17552.000000,16938.000000,17409.000000,13957.000000,17712.000000,17711.000000,14101.000000,5289.000000,5366.000000,5144.000000,5649.000000,4858.000000,6829.000000,6701.000000,6866.000000,6867.000000,6283.000000,6612.000000,6866.000000,6867.000000,6867.000000,6867.000000,6867.000000,6764.000000,6764.000000,6867.000000,6868.000000,6867.000000,6763.000000,6763.000000,6867.000000,6868.000000,6867.000000,6867.000000
mean,126.052918,77.023619,1.612536,1.406683,2.769719,73.228432,18.969465,14.448467,0.687269,15.400410,9.180971,28.616533,4.787577,1.447176,0.957998,75.163383,34.425449,118.602715,0.817138,0.826052,0.916509,0.920244,173.181213,1.592708,13.658928,93.209442,2.268939,28.522060,23.708757,75.251366,6.114405,1.664113,5.278360,70.513161,333.591736,13.246148,4.063509,46.600586,1.724448,3.317938,68.311508,165.125534,24.941238,58.026318,78.209938,15.227648,9.235576,0.196667,214.192261,46.272881,340.020447,30.710417,90.246155,42.041161,145.772537,4.754754,3.714718,0.397313,1.802385,60.325745,6.579688,30.182653,6.097449,2.694441,0.233221,0.165106,0.012576,13.340771,1.134809,10.243145,4.736106,6.258362,143.230240,39.768288,90.603508,30.289684,334.513489,215.117477,1.020532,3.768850,9.410127,2.076115,0.426795,57.697590,30.841913,7.730919,3.513814,1.484792,7.278165,6.435024,1.620080,15.402506,0.192418,9.229942,214.214539,12.151904,39.872089,325.290802,30.152962,91.505928,142.020096,4.791130,0.032268,0.154202,3.520025,0.354889,2.060699,0.547834,2.497921,56.981411,5.831799,34.172333,6.119901
std,19.265082,13.385752,1.335518,0.341184,0.852217,9.008519,12.073726,2.659356,4.187223,5.268567,4.078198,4.205136,9.769779,0.257332,0.245131,24.130157,41.370541,156.473068,0.376352,0.375329,0.276406,0.271478,32.151905,0.540641,7.633927,31.702618,2.660006,25.590902,14.127274,4.846767,0.693130,0.289750,1.335808,18.377230,94.163933,5.517871,2.298907,3.148903,2.616732,5.316967,299.103973,8.689532,112.786636,7.955760,11.329891,3.243238,1.476788,0.130493,55.678726,4.621803,15.235557,2.594384,6.180490,7.384950,17.250828,0.523364,1.277932,0.149948,0.610762,8.814663,1.976304,8.305330,1.616682,2.199247,0.334813,0.161038,0.021400,1.652637,1.393548,8.362729,0.523044,1.614792,16.769623,12.016124,6.050279,2.471872,15.049162,55.691769,0.005982,1.258959,1.791910,0.737653,0.209125,13.335673,10.748792,6.360497,3.364076,1.251164,6.602274,3.90433

In [31]:
merged_train_df['10004'].sort_values(ascending=True)[:5]

21234   -1799.76001
11101       1.19000
10352       1.38000
19802       1.44000
35939       1.52000
Name: 10004, dtype: float32

In [32]:
merged_train_df.loc[21234, '10004'] = np.nan

In [33]:
merged_train_df['2403'].sort_values(ascending=False)[:5]

21196    57142.000000
24805      154.000000
31084      151.100006
27379      150.100006
25556      147.600006
Name: 2403, dtype: float32

In [34]:
merged_train_df.loc[21196, '2403'] = np.nan

In [35]:
merged_train_df['2405'].sort_values(ascending=False)[:5]

21196    21507.000000
32136       54.700001
8087        49.299999
9416        46.500000
24805       45.000000
Name: 2405, dtype: float32

In [36]:
merged_train_df.loc[21196, '2405'] = np.nan

In [37]:
merged_test_df.describe()

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,0424,100005,100006,100007,100014,10002,10003,10004,1106,1107,1115,1117,1127,1321,1322,1325,1326,1345,139,143,1474,155,1814,1815,183,1840,1845,1850,190,191,192,193,2174,2333,2372,2403,2404,2405,2406,2420,269003,269004,269005,269006,269007,269008,269009,269010,269011,269012,269013,269014,269015,269016,269017,269018,269019,269020,269021,269022,269023,269024,269025,300017,300021,31,312,313,314,315,316,317,319,3193,32,320,33,34,37,38,39,669001,669002,669006,809001,809009,979001,979002,979003,979004,979005,979006,979007,979008,979009,979011,979012,979013,979014,979015,979016,979017,979018,979019,979020,979021,979022,979023
count,0.0,0.0,0.0,0.0,0.0,6226.000000,3249.000000,4372.000000,3763.000000,1215.000000,5040.000000,4582.000000,8653.000000,1364.000000,1334.000000,5736.000000,6515.000000,1979.000000,4171.000000,4174.000000,1472.000000,1470.000000,2117.000000,1447.000000,1291.000000,1307.000000,2171.000000,9457.000000,8234.000000,5139.000000,9151.000000,4267.000000,8737.000000,9344.000000,9245.000000,7437.000000,6426.000000,5012.000000,4381.000000,4324.000000,9096.000000,9096.000000,9051.000000,3323.000000,2481.000000,2015.000000,2047.000000,1980.000000,2016.000000,1775.000000,2016.000000,2016.000000,2016.000000,1910.000000,2016.000000,2015.000000,2014.000000,2045.000000,2014.000000,2016.000000,2047.000000,2016.000000,2047.000000,2047.000000,2047.000000,2047.000000,2047.000000,2037.000000,1233.000000,1434.000000,4434.000000,4395.000000,4371.000000,5101.000000,4434.000000,4434.000000,4371.000000,4434.000000,9123.000000,4394.000000,4267.000000,4363.000000,3528.000000,4434.000000,4434.000000,3559.000000,1269.000000,1289.000000,1227.000000,1449.000000,1257.000000,1737.000000,1693.000000,1744.000000,1745.000000,1579.000000,1669.000000,1745.000000,1745.000000,1745.000000,1745.000000,1745.000000,1711.000000,1711.000000,1745.000000,1745.000000,1745.000000,1710.000000,1710.000000,1745.000000,1745.000000,1745.000000,1745.000000
mean,NaN,NaN,NaN,NaN,NaN,73.167038,19.044073,14.465755,0.639326,15.469995,9.177660,28.575964,4.850443,1.456430,0.973028,75.305145,34.064640,111.253899,0.818339,0.833682,0.914293,0.916980,173.189865,1.602104,13.921929,93.510193,2.298941,28.194424,23.566305,75.250336,6.103377,1.669909,5.271622,70.620667,334.480560,13.242046,4.036046,46.645569,1.716293,3.249072,66.660454,165.030075,24.349464,57.921127,78.094032,15.254146,9.242012,0.193128,212.887848,46.146084,340.847229,30.804811,90.268356,42.265617,146.614090,4.771128,3.702415,0.394585,1.812570,60.182590,6.520434,30.334812,6.093307,2.711465,0.231568,0.166135,0.012847,13.347914,1.149616,10.046620,4.739432,6.277516,142.969864,39.841846,90.537903,30.242403,334.134521,214.913849,1.020538,3.783976,9.417546,2.078981,0.427678,57.471344,30.652445,7.576245,3.586543,1.506105,7.762427,6.302767,1.598640,15.372101,0.190882,9.261462,213.016510,12.131538,39.584080,325.339447,30.243736,91.740448,142.100388,4.777490,0.032489,0.151580,3.509603,0.353081,2.054545,0.548614,2.464152,56.967426,5.834905,34.241501,6.097507
std,NaN,NaN,NaN,NaN,NaN,8.987348,12.156971,2.661384,3.875273,5.915576,4.149550,4.207769,1.301552,0.263987,0.262275,23.889549,37.607773,67.720604,0.382742,0.376801,0.263051,0.265523,33.743542,0.515138,8.312960,31.781656,2.493092,22.530043,11.285617,4.879272,0.680129,0.292620,1.316012,20.362608,94.957878,5.475301,2.027271,3.212320,1.426009,3.317561,12.759216,8.877811,3.539409,8.171227,11.128667,3.365330,1.485286,0.051775,54.854378,4.678848,15.683137,2.495205,5.961137,7.080872,16.941759,0.521393,1.212102,0.149875,0.586123,8.728309,1.976968,8.167378,1.540953,2.205780,0.298584,0.166724,0.022571,1.686410,1.402315,8.324814,0.515019,1.608065,16.413521,11.874910,6.003174,2.464576,14.952880,55.968987,0.006012,1.250886,1.791144,0.733974,0.203998,13.934575,10.821147,6.206759,2.795492,1.134165,8.325024,3.757018,0.740983,2.341707,0.047923,1.512505,58.509899,1.084208,9.361670,40.298973,2.413367,6.431439,23.411612,0.520832,0.035170,0.146861,1.173711,0.121018

In [38]:
merged_train_df.to_csv('data_train.csv')
merged_test_df.to_csv('data_test.csv')